### Create sample and load pre-token

In [ ]:
full_text = "low low low low low lower lower widest widest widest newest newest newest newest newest newest"

freq_pre_token = {}
for p in full_text.split(" "):
    if p not in freq_pre_token: 
        freq_pre_token[p] = 1
    else:
        freq_pre_token[p] += 1

freq_pre_token
print(f"Get key low: 'low': {freq_pre_token['low']}")
freq_pre_token = {tuple(k.encode('utf-8')): v for k, v in freq_pre_token.items()}

print(f' low encode to: {[i for i in "low".encode("utf-8")]}')
print(f"Get key low: 'low': {freq_pre_token[tuple('low'.encode('utf-8'))]}")
freq_pre_token

In [ ]:
# create vocab with 256 bytes values and <|endoftext|>
index_2_vocab = {i: bytes([i]) for i in range(256)}
index_2_vocab[256] = '<|endoftext|>'

index_2_vocab[108] + index_2_vocab[111] + index_2_vocab[119]

In [ ]:
index_2_vocab[108]

In [ ]:
# create linked list of each pre-token to tracking and update pair
class Node:
    def __init__(self, vocab_index, n_count=0):
        self.vocab_index = vocab_index
        self.next = None
        self.prev = None
        self.n_count = n_count

    def __repr__(self):
        return str(index_2_vocab[self.prev.vocab_index] if self.prev else None) + "-" + str(index_2_vocab[self.vocab_index]) + "-" + str(index_2_vocab[self.next.vocab_index] if self.next else None) + " (" + str(self.n_count) + ")"


freq_linked_list = {}

for k, v in freq_pre_token.items():
    nodes = tuple([Node(i, v) for i in k])
    for pre, n in zip(nodes, nodes[1:]):
        pre.next = n
        n.prev = pre
    freq_linked_list[nodes] = v


freq_linked_list

In [ ]:
import heapq
# init the pair_count

# pair_count is a dict, with key is pair of bytes, and value is the list of pair node.
# pair_version is a dict, with key is pair of bytes, and value is the version of the pair. Used to check out of date pair.
# pair_max_heap is max heap of pair_count, 

pair_version = {}
pair_count = {}
max_heap = []

for ns, v in freq_linked_list.items():
    for n, next_n in zip(ns, ns[1:]):
        pair_bytes = index_2_vocab[n.vocab_index] + index_2_vocab[next_n.vocab_index]
        n_count = n.n_count

        # update pair_count
        if pair_bytes not in pair_count:
            pair_count[pair_bytes] = {'n_count': n_count, 'pair_nodes': [(n, next_n)]}
        else:
            pair_count[pair_bytes]['n_count'] += n_count
            pair_count[pair_bytes]['pair_nodes'].append((n, next_n))

        # update pair version
        if pair_bytes not in pair_version:
            pair_version[pair_bytes] = 1
        else:
            pair_version[pair_bytes] += 1   

        # update max_heap
        heapq.heappush(max_heap, (-pair_count[pair_bytes]['n_count'], pair_bytes, pair_version[pair_bytes]))

max_heap

In [ ]:
# get max pair
max_pair = heapq.heappop(max_heap)

In [ ]:
# merge max pair -> update pair_count -> update max_heap -> update pair_version
max_pair

In [ ]:
pair_count